# Analysis of the Milos Zeman's speeches

### Import packages

In [1]:
from time import sleep  # sleeping
from requests import get  # downloading page source
from bs4 import BeautifulSoup  # BeautifulSoup for processing html source pages
from pandas import DataFrame  # DataFrame for storing and manipulating data
from pandas import to_datetime  # convert string to datetime
from selenium.common.exceptions import UnexpectedAlertPresentException  # selenium alert exception
from selenium.webdriver import Chrome  # Chrome browser driver

## Download list of articles

#### First download webpage with all articles
All speeches are listed on the [Milos Zeman's webpage](http://www.zemanmilos.cz/cz/clanky). Initialy there are only few articles listed. We must dynamicly list all articles by clicking **NAČÍST DALŠÍ ČLÁNKY** button till possible. Then we extract html source code of the page

In [2]:
def click(id_, driver, sleep_time=0.3):
    """
    Clicks button with given id.
    Args:
        id_: id of the buttton
        driver: browser
        sleep_time: sleeps twice (after finding button and after clicking)
    """
    button = driver.find_element_by_id(id_=id_)  # find button
    sleep(sleep_time)  # sleep
    button.click()  # click
    sleep(sleep_time)  # sleep
    
def download_page_with_articles(url):
    """
    Clicks on button 'dalsistrankabtn' until all articles listed.
    Args:
        url: url to page with list of articles
    Returns:
        page_source: html source of whole page
    """
    print('-'*50)
    print('Opening browser...\n')
    browser = Chrome(executable_path="./chromedriver")  # open browser
    browser.get(url)  # get page

    while True:
        try:
            # click on button for next articles
            click(id_='dalsistrankabtn', driver=browser)
        except UnexpectedAlertPresentException:  # if all articles listed
            alert = browser.switch_to_alert()
            alert.accept()  # accept the alert window
            break
        except Exception as e:
            print(e)
            sleep(30)
    page_source = browser.page_source  # get source page
    browser.quit()
    print('\nClosing browser.')
    print('-'*50)
    return page_source

URL = 'http://www.zemanmilos.cz/cz/clanky/'  # url to milos zeman's official webpage
page_source = download_page_with_articles(url=URL)

------------------------------
Opening browser.
Message: unknown error: Element <a class="read_more_big center mouse" id="dalsistrankabtn" onclick="newsAjax('clanky');" title="..." tabindex="1">NAČÍST DALŠÍ ČLÁNKY</a> is not clickable at point (368, 698). Other element would receive the click: <h3>...</h3>
  (Session info: chrome=58.0.3029.110)
  (Driver info: chromedriver=2.29.461585 (0be2cd95f834e9ee7c46bcc7cf405b483f5ae83b),platform=Mac OS X 10.12.5 x86_64)

Closing browser.
------------------------------


#### Process webpage and get links to all articles
Now we find all speeches, i.e. articles with **"Projev"** in the title. We save title of the speech, date the speech was published and link that leads to full text of the speech.

In [4]:
def process_page(page_source):
    """
    From source page with all articles get title, date, link
    Args:
        page_source: html source of the page with articles
    Returns:
        df: pandas DataFrame with columns=['date', 'link', 'title']
    """
    page = BeautifulSoup(page_source, 'lxml')  # process

    # find all articles
    articles = page.find_all(name='div', attrs={'class': 'news_view'})
    
    df = DataFrame(columns=['date', 'link', 'title'])  # create DataFrame
    speech_index = 0
    for article in articles:  # for all listed articles
        # date article was published
        date = article.span.text[-10:]
        # link to article
        link = 'http://www.zemanmilos.cz/' + article.h3.a['href']
        # title of the article
        title = article.h3.a.text

        # if the article is 'Projev' and has acceptable data
        if 'Projev' in title and int(date[-4:]) > 0:
            new_row = DataFrame(data={'date': date,
                                      'link': link,
                                      'title': title},
                                index=[speech_index])
            df = df.append(new_row)  # add to DataFrame
            speech_index += 1

    # parse date (only date, without time)
    df['date'] = to_datetime(arg=df['date'], format='%d.%M.%Y').dt.floor('d')
    return df

df = process_page(page_source=page_source)
df.head(10)  # show DataFrame

,date,link,title
0,2017-01-16,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při udělení vyznam...
1,2017-01-16,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...
2,2017-01-14,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...
3,2017-01-12,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání se zas...
4,2017-01-07,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...
5,2017-01-06,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky na Česko-vietnamsk...
6,2017-01-31,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při zahájení velet...
7,2017-01-29,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání se čle...
8,2017-01-27,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při převzetí symbo...
9,2017-01-24,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání s tisk...


### Download each speech
Now we visit webpage of every listed article, select only plain text of the speech and save it to DataFrame.

In [7]:
def download_articles(df):
    """
    Download text of each article.
    Args:
        df: pandas DataFrame with columns=['date', 'link', 'title']
    Returns:
        df: pandas DataFrame with columns=['date', 'link', 'title', 'text']
    """
    print('-'*50)
    print('Downloading each article...\n')
    speeches = []
    for speech_link in df['link'].values:  # for every link to article
        try:
            # get article source page
            req = get(url=speech_link).text.encode(encoding='utf8')
            # convert to html
            page = BeautifulSoup(markup=req, features='lxml')
            # find text of the speech
            speech = page.find(name='div', attrs={'class': 'wrap_detail'}).text
        except Exception as e:  # if exception occurs
            print('Exception for link {:s}:'.format(speech_link),  e)
        speech = speech.replace(u'\n', u'')  # replace newline tag
        speech = speech.replace(u'\xa0', u' ')  # replace hard space tag
        speeches.append(speech)
    df['text'] = speeches  # add to DataFrame
    print('\nArticles downloaded.')
    print('-'*50)
    return df
df = download_articles(df=df)

# save to csv
df.to_csv('./speeches.csv', encoding='utf8', index=False, sep=';')

df.head(10)  # show DataFrame

--------------------------------------------------

Exception for link http://www.zemanmilos.cz/cz/clanky/projev-prezidenta-republiky-na-14-rocniku-mezinarodni-konference--investment-&-business-forum--480054.htm: 'NoneType' object has no attribute 'text'

Articles downloaded.
--------------------------------------------------


,date,link,title,text
0,2017-01-16,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při udělení vyznam...,"Dámy a pánové,dovolte mi, abych úvodem tlumoči..."
1,2017-01-16,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...,Protože naše setkání je po audienci u královn...
2,2017-01-14,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...,"Vážená paní hejtmanko, dámy a pánové,předevší..."
3,2017-01-12,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání se zas...,"Vážená paní hejtmanko, vážení členové krajské..."
4,2017-01-07,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při tiskové konfer...,"Vážený pane prezidente, dámy a pánové, dovo..."
5,2017-01-06,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky na Česko-vietnamsk...,"Vážení vietnamští přátelé, myslím, že si moh..."
6,2017-01-31,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při zahájení velet...,"Vážený pane ministře, vážený pane hejtmane, dá..."
7,2017-01-29,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání se čle...,"Vážený hejtmane, vážení členové krajského zas..."
8,2017-01-27,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při převzetí symbo...,"Vážená paní starostko, milí spoluobčané,rád j..."
9,2017-01-24,http://www.zemanmilos.cz/cz/clanky/projev-prez...,Projev prezidenta republiky při setkání s tisk...,"Vážený pane prezidente, dámy a pánové,jsem ve..."
